In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
from astropy import units
import os
import astropy.units as u
import matplotlib.pyplot as plt

import snapshot_obj
import dataset_compute

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(dataset_compute)

# Radial distribution of satellites in the Local Group

Count the accumulation of satellites with radius from the centres of potential of the MW and M31 galaxies **together**.

Create a dictionary of the datasets from each simulation. 

In [ ]:
snap_id = 127
sim_ids = ["V1_MR_fix", "V1_MR_curvaton_p082_fix"]
names = ["plain-LCDM", "curv-p082"]

# Define M31 and MW in each simulation:
m31 = [(1,0), (1,0)]
mw = [(2,0), (1,1)]

data = {}
for name, sim_id, m31_ns, mw_ns in zip(names, sim_ids, m31, mw):
    data[name] = {"snapshot": snapshot_obj.Snapshot(sim_id, snap_id, name=name),
                  "M31_identifier": m31_ns,
                  "MW_identifier": mw_ns}

## Get data for plotting

Read the datasets from each simulation, convert the units and split into satellites/isolated and luminous/dark:

In [ ]:
def arrange_radius_array(r):
    r = r * units.cm.to(units.kpc)
    r = np.concatenate((np.sort(r), np.array([10000]))) 
    return r

def construct_count_array(size):
    return np.concatenate((np.arange(1, size+1), 
                           np.array([size])))

In [ ]:
distinction = "By_r"

for key, sim_data in data.items():    
    # Get data:
    snap = sim_data["snapshot"]
    cops = snap.get_subhalos("CentreOfPotential")
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    
    # Compute masking arrays:
    mask_lum, mask_dark = dataset_compute.split_luminous(snap)
    mask_nonzero_vmax = dataset_compute.prune_vmax(snap)
    if distinction == "By_r":
        masks_sat, mask_isol = dataset_compute.split_satellites_by_distance(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    elif distinction == "By_GN":
        masks_sat, mask_isol = dataset_compute.split_satellites_by_group_number(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    
    mask_m31_lum = np.logical_and.reduce([masks_sat[0], mask_lum, mask_nonzero_vmax])
    mask_m31_dark = np.logical_and.reduce([masks_sat[0], mask_dark, mask_nonzero_vmax])
    mask_mw_lum = np.logical_and.reduce([masks_sat[1], mask_lum, mask_nonzero_vmax])
    mask_mw_dark = np.logical_and.reduce([masks_sat[1], mask_dark, mask_nonzero_vmax])
    mask_isol_lum = np.logical_and.reduce([mask_isol, mask_lum, mask_nonzero_vmax])
    mask_isol_dark = np.logical_and.reduce([mask_isol, mask_dark, mask_nonzero_vmax])
    
    # In case M31 and MW are identified as the same halo, remove intersection:
    mask_mw_lum = np.logical_and(mask_mw_lum, np.logical_not(mask_m31_lum))
    mask_mw_dark = np.logical_and(mask_mw_dark, np.logical_not(mask_m31_dark))
   
    # Compute distances to M31 and MW:
    m31_cop = cops[snap.index_of_halo(sim_data["M31_identifier"][0],
                                      sim_data["M31_identifier"][1])]
    dist_to_m31 = dataset_compute.distance_to_point(snap, m31_cop)\
                     * units.cm.to(units.kpc)
    mw_cop = cops[snap.index_of_halo(sim_data["MW_identifier"][0],
                                     sim_data["MW_identifier"][1])]
    dist_to_mw = dataset_compute.distance_to_point(snap, mw_cop)\
                     * units.cm.to(units.kpc)

    # Add luminous satellites to the data dictionary:
    dist_lum = np.concatenate([dist_to_m31[mask_m31_lum], dist_to_mw[mask_mw_lum]])
    vmax_lum = np.concatenate([vmax[mask_m31_lum], vmax[mask_mw_lum]])
    sort_idx = np.argsort(dist_lum)
    sim_data["satellites"] = {
        "radius" : 
        {"luminous" : np.concatenate([dist_lum[sort_idx], np.array([10000])])},
        "Count" :
        {"luminous" : np.arange(1, dist_lum.size+2)},
        "Vmax" : 
        {"luminous" : np.concatenate([vmax_lum[sort_idx], np.array([0.01])])}
    }
    
    # Add dark satellites to the data dictionary:
    dist_dark = np.concatenate([dist_to_m31[mask_m31_dark], dist_to_mw[mask_mw_dark]])
    vmax_dark = np.concatenate([vmax[mask_m31_dark], vmax[mask_mw_dark]])
    sort_idx = np.argsort(dist_dark)
    sim_data["satellites"]["radius"].update( 
        {"dark" : np.concatenate([dist_dark[sort_idx], np.array([10000])])}
    )
    sim_data["satellites"]["Count"].update(
        {"dark" : np.arange(1, dist_dark.size+2)}
    )
    sim_data["satellites"]["Vmax"].update(
        {"dark" : np.concatenate([vmax_dark[sort_idx], np.array([0.01])])}
    )
    
    # Compute distances to LG centre:
    lg_centre = dataset_compute.compute_LG_centre(snap,
                                                  sim_data["M31_identifier"],
                                                  sim_data["MW_identifier"])
    dist_to_lg = dataset_compute.distance_to_point(snap, lg_centre)\
                    * units.cm.to(units.kpc)
    
    # ... and add isolated galaxies to the data dictionary:
    dist_lum = dist_to_lg[mask_isol_lum]
    vmax_lum = vmax[mask_isol_lum]
    sort_idx = np.argsort(dist_lum)
    sim_data["isolated"] = {
        "radius" : 
        {"luminous" : np.concatenate([dist_lum[sort_idx], np.array([10000])])},
        "Count" :
        {"luminous" : np.arange(1, dist_lum.size+2)},
        "Vmax" : 
        {"luminous" : np.concatenate([vmax_lum[sort_idx], np.array([0.01])])}
    }
    
    dist_dark = dist_to_lg[mask_isol_dark]
    vmax_dark = vmax[mask_isol_dark]
    sort_idx = np.argsort(dist_dark)
    sim_data["isolated"]["radius"].update(
        {"dark" : np.concatenate([dist_dark[sort_idx], np.array([10000])])}
    )
    sim_data["isolated"]["Count"].update(
        {"dark" : np.arange(1, dist_dark.size+2)}
    )
    sim_data["isolated"]["Vmax"].update(
        {"dark" : np.concatenate([vmax_dark[sort_idx], np.array([0.01])])}
    )
    

## Plot

In [ ]:
# Set some parameters:
x_down_sat = 0; x_up_sat = 400
y_down_sat = 1; y_up_sat = 5000

x_down_isol = 0; x_up_isol = 2100
y_down_isol = 1; y_up_isol = 10000

# Set colors:
color = ["black", "red", "blue", "green"]

In [ ]:
# Construct saving location:
filename = 'sat_and_isol_r_dist_{}'.format(distinction)
for name in names:
    filename += "_{}".format(name)
filename += ".png"
    
home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures", "MediumResolution", "CumulativeDistributions")
filename = os.path.join(path, filename)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(14,6))
plt.subplots_adjust(wspace=0.3)

# Set axis for satellites:
axes[0].set_xlim(x_down_sat, x_up_sat)
axes[0].set_ylim(y_down_sat, y_up_sat)
axes[0].set_xlabel('$r_\mathrm{central}[\mathrm{kpc}]$', fontsize=16)
axes[0].set_ylabel('$N(<r_\mathrm{central})$', fontsize=16)
axes[0].set_yscale('log')
axes[0].yaxis.set_ticks_position("both")

# Set axis for isolated galaxies:
axes[1].set_xlim(x_down_isol, x_up_isol)
axes[1].set_ylim(y_down_isol, y_up_isol)
axes[1].set_xlabel('$r_\mathrm{LG}}[\mathrm{kpc}]$', fontsize=16)
axes[1].set_ylabel('$N(<r_\mathrm{LG})$', fontsize=16)
axes[1].set_yscale('log')
axes[1].yaxis.set_ticks_position("both")
    
axes[0].set_title('Satellite galaxies')
axes[1].set_title('Isolated galaxies')

# Add scatter plots:
for i, (name, entry) in enumerate(data.items()):
    x = entry['satellites']['radius']['luminous']
    y = entry['satellites']['Count']['luminous']
    axes[0].scatter(x, y, c=color[i], label='{} luminous'.format(name))
    x = entry['satellites']['radius']['dark']
    y = entry['satellites']['Count']['dark']
    axes[0].scatter(x, y, c=color[i], linestyle='--', label='{} dark'.format(name))
    
    x = entry['isolated']['radius']['luminous']
    y = entry['isolated']['Count']['luminous']
    axes[1].plot(x, y, c=color[i], label='{} luminous'.format(name))
    x = entry['isolated']['radius']['dark']
    y = entry['isolated']['Count']['dark']
    axes[1].plot(x, y, c=color[i], linestyle='--', label='{} dark'.format(name))

axes[0].legend(loc='upper right')
plt.tight_layout()

plt.savefig(filename, dpi=200)

In [ ]:
fig, axes = plt.subplots(ncols=2, sharey="row", figsize=(14,6))
plt.subplots_adjust(wspace=0.3)

# Set some parameters:
x_down_sat = 0; x_up_sat = 300
x_down_isol = 0; x_up_isol = 2000
y_down = 1; y_up = 500

# Set axis for satellites:
axes[0].set_xlim(x_down_sat, x_up_sat)
axes[0].set_ylim(y_down, y_up)
axes[0].set_xlabel('$r_\mathrm{central}[\mathrm{kpc}]$')
axes[0].set_ylabel('$N(<r)$')
axes[0].set_yscale('log')
axes[0].yaxis.set_ticks_position("both")

# Set axis for isolated galaxies:
axes[1].set_xlim(x_down_isol, x_up_isol)
axes[1].set_xlabel('$r_\mathrm{LG}}[\mathrm{kpc}]$')
axes[1].set_yscale('log')
axes[1].yaxis.set_ticks_position("both")
    
axes[0].set_title('Satellite galaxies')
axes[1].set_title('Isolated galaxies')

smin = 1; smax = 30
masses = data["plain-LCDM"]["Vmax_M31"][data["plain-LCDM"]["Selections"]["M31"]["Luminous"]]
mmax = np.min(masses)
mmin = np.max(masses)
    
def transf(x):
    return 1/(mmax-mmin) * (smin*(x-mmin) - smax*(x-mmax))

# Add scatter plots:
offset = 0.03
for i, (name, entry) in enumerate(data.items()):
    
    r = entry['satellites']['radius']['luminous']
    cnt = entry['satellites']['Count']['luminous']
    
    mask_lum = np.logical_or(entry["Selections"]["M31"]["Luminous"],
                             entry["Selections"]["MW"]["Luminous"])
    r = entry["R_M31"]
    vmax = entry["Vmax_M31"]
    axes[0].plot(r[mask_lum], cnt_lum, 
                 c=color[i], alpha=0.3, linestyle="solid",
                 label='{} luminous'.format(name))
    axes[0].scatter(r[mask_lum], cnt_lum * 10**offset, s=transf(vmax[mask_lum]),
                    facecolors="none", edgecolors=color[i], alpha=0.7)
    
    mask_dark = np.logical_or(entry["Selections"]["M31"]["Dark"],
                              entry["Selections"]["MW"]["Dark"])
    r = entry["R_M31"]
    vmax = entry["Vmax_M31"]
    cnt_dark = np.arange(1, np.sum(mask_dark) + 1) 
    axes[0].plot(r[mask_dark], cnt_dark, 
                 c=color[i], alpha=0.3, linestyle="dashed",
                 label='{} luminous'.format(name))
    axes[0].scatter(r[mask_dark], cnt_dark * 10**offset, s=transf(vmax[mask_dark]),
                    c=color[i], alpha=0.7)
    
    
    mask_lum = entry["Selections"]["Isolated"]["Luminous"]
    r = entry["R_LG"]
    vmax = entry["Vmax_LG"]
    cnt_lum = np.arange(1, np.sum(mask_lum) + 1) 
    axes[1].plot(r[mask_lum], cnt_lum, 
                 c=color[i], alpha=0.3, linestyle="solid",
                 label='{} luminous'.format(name))
    axes[1].scatter(r[mask_lum], cnt_lum * 10**offset, s=transf(vmax[mask_lum]),
                    facecolors="none", edgecolors=color[i], alpha=0.7)
    
    mask_dark = entry["Selections"]["Isolated"]["Dark"]
    r = entry["R_LG"]
    vmax = entry["Vmax_LG"]
    cnt_dark = np.arange(1, np.sum(mask_dark) + 1) 
    axes[1].plot(r[mask_dark], cnt_dark, 
                 c=color[i], alpha=0.3, linestyle="dashed",
                 label='{} luminous'.format(name))
    axes[1].scatter(r[mask_dark], cnt_dark * 10**offset, s=transf(vmax[mask_dark]),
                    c=color[i], alpha=0.7)
    
#     x = entry['satellites']['radius']['dark']
#     y = entry['satellites']['Count']['dark']
#     axes[0].scatter(x, y, c=color[i], linestyle='--', label='{} dark'.format(name))
    
#     x = entry['isolated']['radius']['luminous']
#     y = entry['isolated']['Count']['luminous']
#     axes[1].plot(x, y, c=color[i], label='{} luminous'.format(name))
#     x = entry['isolated']['radius']['dark']
#     y = entry['isolated']['Count']['dark']
#     axes[1].plot(x, y, c=color[i], linestyle='--', label='{} dark'.format(name))

# axes[0].legend(loc='upper right')
plt.tight_layout()

plt.savefig("testi.png", dpi=200)
# plt.savefig(filename, dpi=200)